In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Oct_dict_JSON.json', 'r') as json_file:
    link_day_minute_Oct_dict_JSON = json.load(json_file)

In [10]:
week_day_Oct_list = [1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 15, 16, 17, 18, 19, 22, 23, 24, 25, 26, 29, 30, 31]

In [11]:
link_day_minute_Oct_list = []
for link_idx in range(24):
    for day in week_day_Oct_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Oct_list.append(link_day_minute_Oct_dict_JSON[key] ['PM_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Oct_list)

66240

In [13]:
x = np.matrix(link_day_minute_Oct_list)
x = np.matrix.reshape(x, 24, 2760)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2760)

In [15]:
x[:,:2]

matrix([[ 334120.96216501,  340716.07800673],
        [ 347248.15225179,  349083.0435739 ],
        [ 621097.23609129,  629786.08225115],
        [ 663175.62453404,  673045.94651344],
        [ 168752.25797173,  171620.17689842],
        [ 201242.83650981,  198921.77361329],
        [ 405493.2749045 ,  407176.90029079],
        [ 403933.49606626,  402938.1620441 ],
        [ 394939.65509079,  387093.16161759],
        [ 409098.88861844,  403360.66899784],
        [ 411894.20808558,  429938.0014987 ],
        [ 333850.55711395,  364146.6240118 ],
        [  93773.91087962,   92723.35338757],
        [  99446.3481581 ,  102264.61004274],
        [ 165332.43420605,  164872.05787731],
        [ 131057.45664589,  139977.42080595],
        [ 210756.24845907,  211272.78568787],
        [ 212839.8838141 ,  213874.84436637],
        [ 367889.52880085,  366968.8690794 ],
        [ 348342.84561268,  355022.56346719],
        [ 524961.08385964,  526232.78711439],
        [ 568204.35288183,  562315

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [3e-01, 1e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.01s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -9.75238981e+06  0.00000000e+00  4.70e+03 1.33e+03  1.00e+06     0s
   1  -3.29775362e+06 -4.19116004e+06  6.64e+02 1.88e+02  1.90e+05     0s
   2  -2.69618505e+06 -4.28905659e+06  3.19e+01 9.03e+00  3.60e+04     0s
   3  -3.20446761e+06 -3.47421339e+06  3.19e-05 9.03e-06  4.8

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Oct_weekday_PM.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[1.6227166473717599e-09,
 183366.00822247053,
 39784.58411996962,
 1.611523748778917e-09,
 121717.95557912078,
 2.984720636079246e-09,
 2.945322230940867e-09,
 1.3690437888607345e-08,
 9.641442239179514e-11,
 139018.3736016918,
 9.097175414712696e-11,
 9.641442239260247e-11,
 9.339743654887994e-11,
 9.397531672280053e-11,
 396890.06234546815,
 5.008949166347016e-10,
 1.611523750118076e-09,
 1.6115237482571447e-09,
 428647.73653190455,
 2.986563024893321e-09,
 2.9321804003990987e-09,
 96522.62524572416,
 139885.9729553164,
 1.6935195571574275e-07,
 1.6930766048368255e-07,
 9.547330299381498e-10,
 6451.466439412697,
 101840.55861192376,
 0.3770757480719453,
 5.008949157272117e-10,
 52849.364788813764,
 51756.97147825225,
 9.601473618089924e-10,
 174188.6412555736,
 53047.243089784795,
 5.067948680173573e-05,
 5.008899647081443e-10,
 208112.9446474641,
 6.619409075878127e-10,
 6.619409072817162e-10,
 2.1602800754623268e-08,
 1.9158295906294525e-08,
 9.258103932447337e-07,
 5.0062406210273